# Preprocessing(1)

## Importing Necessary Libraries

In [1]:
#!pip install plotly

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
import re
import plotly.figure_factory as ff
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.preprocessing import MultiLabelBinarizer


## Loading Data

In [3]:
df = pd.read_csv('games_info.csv')
game_details = pd.read_csv('games_name_url_price.csv')

## Initial Data Exploration

### games_info.csv

In [4]:
df.head()

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,SUPPORTED_PLATFORMS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,RELEASE_DATE,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK
0,Dragon's Dogma: Dark Arisen,"Store Genres Action (1), Adventure (25), RPG (3)",review_score 8,7 Languages,Capcom,Windows,39271,5015,NaN,15 January 2016 – 16:48:00 UTC (9 years ago),"27,368\nall-time peak 9 years ago17 January 2016",$22.49,3.0,"1,064\n24-hour peak"
1,Forza Horizon 5,"Store Genres Action (1), Adventure (25), Racin...",review_score 8,24 Languages,Playground Games,Windows,156683,21075,NaN,9 November 2021 – 04:58:00 UTC (3 years ago),"81,096\nall-time peak 3 years ago13 November 2021",$32.78,53.0,"16,131\n24-hour peak"
2,Thrive,"Store Genres Casual (4), Indie (23), Simulatio...",review_score 8,1 Languages,Revolutionary Games Studio,Windows Linux,872,70,Engine.Godot,26 November 2021 – 10:54:48 UTC (3 years ago),145\nall-time peak 2 years ago24 July 2022,$2.99,NaN,21\n24-hour peak
3,Layers of Fear (2016),"Store Genres Adventure (25), Indie (23)",review_score 8,12 Languages,Bloober Team SA,Windows macOS Linux,22766,2819,Engine.Unity,15 February 2016 – 21:53:00 UTC (8 years ago),"145,384\nall-time peak 6 years ago13 June 2018",$10.19,3.0,31\n24-hour peak
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",review_score 8,1 Languages,lazarche,Windows,834,66,"Detected Technologies (?), SDK.SteamworksNET",25 May 2023 – 10:22:41 UTC (12 months ago),"3,778\nall-time peak 10 months ago18 July 2023",NaN,4.0,318\n24-hour peak


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259 entries, 0 to 2258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2259 non-null   object 
 1   STORE_GENRE            2221 non-null   object 
 2   RATING_SCORE           2225 non-null   object 
 3   N_SUPPORTED_LANGUAGES  2226 non-null   object 
 4   DEVELOPERS             2258 non-null   object 
 5   SUPPORTED_PLATFORMS    2197 non-null   object 
 6   POSITIVE_REVIEWS       2259 non-null   int64  
 7   NEGATIVE_REVIEWS       2259 non-null   int64  
 8   TECHNOLOGIES           1845 non-null   object 
 9   RELEASE_DATE           2255 non-null   object 
 10  TOTAL_TWITCH_PEAK      2259 non-null   object 
 11  PRICE                  1507 non-null   object 
 12  N_DLC                  789 non-null    float64
 13  24_HOUR_PEAK           2208 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 247.2+

In [6]:
df.describe()

,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,N_DLC
count,2.259000e+03,2259.000000,789.000000
mean,2.087847e+04,1270.529438,5.904943
std,7.540080e+04,5824.763668,17.345104
min,3.400000e+01,0.000000,1.000000
25%,7.245000e+02,33.000000,1.000000
50%,2.509000e+03,136.000000,2.000000
75%,1.043100e+04,602.000000,4.000000
max,1.257272e+06,190953.000000,379.000000


In [7]:
df.describe(include='object')

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,SUPPORTED_PLATFORMS,TECHNOLOGIES,RELEASE_DATE,TOTAL_TWITCH_PEAK,PRICE,24_HOUR_PEAK
count,2259,2221,2225,2226,2258,2197,1845,2255,2259,1507,2208
unique,2088,325,3,36,1668,6,234,2071,2066,125,643
top,TOEM,"Store Genres Action (1), Adventure (25), Indie...",review_score 8,1 Languages,Square Enix,Windows,Engine.Unity,17 September 2021 – 13:00:40 UTC (3 years ago),2 years ago,$19.99,4\n24-hour peak
freq,4,113,1355,583,16,1294,764,4,8,206,81


### game_details.csv

In [8]:
game_details.head()

,name,url,price,release,page
0,The Price Of Flesh,https://steamdb.info/app/1940040/,$14.99,2022,38
1,Terminator: Resistance,https://steamdb.info/app/954740/,$39.99,2019,38
2,Ken Follett's The Pillars of the Earth,https://steamdb.info/app/234270/,$19.99,2017,38
3,Subsurface Circular,https://steamdb.info/app/676820/,$4.79,2017,38
4,Sumire,https://steamdb.info/app/1335230/,$5.99,2021,38


In [9]:
game_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4950 entries, 0 to 4949
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     4950 non-null   object
 1   url      4950 non-null   object
 2   price    4950 non-null   object
 3   release  4950 non-null   object
 4   page     4950 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 193.5+ KB


In [10]:
game_details.describe(include='object')

,name,url,price,release
count,4950,4950,4950,4950
unique,4236,4238,191,29
top,Black Book,https://steamdb.info/app/80/,-,2023
freq,3,3,1131,653


## Removing Duplicated Games

In [11]:
df_copy = df.copy()
duplicate_rows = df[df.duplicated(subset=['NAME'])]
print("Duplicate Rows based on the 'NAME' column:")

num_duplicates = duplicate_rows.shape[0]
print(f"Number of duplicate rows based on the 'NAME' column: {num_duplicates}")
df = df_copy.drop_duplicates(subset=['NAME'], keep='first')


Duplicate Rows based on the 'NAME' column:
Number of duplicate rows based on the 'NAME' column: 171


In [12]:
duplicate_rows['NAME'].head()

61                             The Night of the Scissors
132                 If My Heart Had Wings -Flight Diary-
133                                              KNIGHTS
139    Command & Conquer: Red Alert™ 2 and Yuri’s Rev...
156                                         Metro Exodus
Name: NAME, dtype: object

In [13]:
del df_copy
del duplicate_rows

## Handling Missing Values

In [14]:
df.replace('N/A', pd.NA, inplace=True)

In [15]:
def missing_percentage(df):
    missing_percentage = df.isnull().mean() * 100
    missing_percentage_table = pd.DataFrame({'Column': missing_percentage.index, 'Missing Percentage': missing_percentage.values})
    missing_percentage_table = missing_percentage_table.sort_values(by='Missing Percentage', ascending=False)
    return missing_percentage_table

missing_percentage_table = missing_percentage(df)

In [16]:
missing_percentage_table

,Column,Missing Percentage
12,N_DLC,65.038314
11,PRICE,32.950192
8,TECHNOLOGIES,18.582375
5,SUPPORTED_PLATFORMS,2.729885
13,24_HOUR_PEAK,2.250958
1,STORE_GENRE,1.724138
2,RATING_SCORE,1.532567
3,N_SUPPORTED_LANGUAGES,1.484674
9,RELEASE_DATE,0.191571
4,DEVELOPERS,0.047893


In [17]:
del missing_percentage_table

In [18]:
def drop_rows_with_missing_data(df, threshold_percentage):
    threshold = len(df.columns) * (threshold_percentage / 100)
    initial_row_count = len(df)
    df_cleaned = df.dropna(thresh=len(df.columns) - threshold)
    final_row_count = len(df_cleaned)
    rows_deleted = initial_row_count - final_row_count
    return df_cleaned, rows_deleted

df, rows_deleted = drop_rows_with_missing_data(df, 25)

In [19]:
print(f"Number of rows deleted: {rows_deleted}")

Number of rows deleted: 37


In [20]:
missing_percentage_table = missing_percentage(df)
missing_percentage_table

,Column,Missing Percentage
12,N_DLC,64.895173
11,PRICE,33.008289
8,TECHNOLOGIES,18.430034
5,SUPPORTED_PLATFORMS,2.340322
13,24_HOUR_PEAK,0.731351
1,STORE_GENRE,0.195027
9,RELEASE_DATE,0.195027
4,DEVELOPERS,0.048757
0,NAME,0.000000
2,RATING_SCORE,0.000000


In [21]:
del missing_percentage_table

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2051 entries, 0 to 2258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2051 non-null   object 
 1   STORE_GENRE            2047 non-null   object 
 2   RATING_SCORE           2051 non-null   object 
 3   N_SUPPORTED_LANGUAGES  2051 non-null   object 
 4   DEVELOPERS             2050 non-null   object 
 5   SUPPORTED_PLATFORMS    2003 non-null   object 
 6   POSITIVE_REVIEWS       2051 non-null   int64  
 7   NEGATIVE_REVIEWS       2051 non-null   int64  
 8   TECHNOLOGIES           1673 non-null   object 
 9   RELEASE_DATE           2047 non-null   object 
 10  TOTAL_TWITCH_PEAK      2051 non-null   object 
 11  PRICE                  1374 non-null   object 
 12  N_DLC                  720 non-null    float64
 13  24_HOUR_PEAK           2036 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 240.4+ KB


## Handle Missing Values:(`N_DLC`)   
### (Remove Columns with High Missing Values)

In [23]:
# df['N_DLC'] = pd.to_numeric(df['N_DLC'], errors='coerce') 
# df['N_DLC'] = df['N_DLC'].fillna(-1)  
# df['N_DLC'] = df['N_DLC'].astype(int) 
# df['N_DLC'] = df['N_DLC'].replace(-1, pd.NA)
# df['N_DLC']

In [24]:
if 'N_DLC' in df.columns:
    df.drop(columns=['N_DLC'], inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2051 entries, 0 to 2258
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   NAME                   2051 non-null   object
 1   STORE_GENRE            2047 non-null   object
 2   RATING_SCORE           2051 non-null   object
 3   N_SUPPORTED_LANGUAGES  2051 non-null   object
 4   DEVELOPERS             2050 non-null   object
 5   SUPPORTED_PLATFORMS    2003 non-null   object
 6   POSITIVE_REVIEWS       2051 non-null   int64 
 7   NEGATIVE_REVIEWS       2051 non-null   int64 
 8   TECHNOLOGIES           1673 non-null   object
 9   RELEASE_DATE           2047 non-null   object
 10  TOTAL_TWITCH_PEAK      2051 non-null   object
 11  PRICE                  1374 non-null   object
 12  24_HOUR_PEAK           2036 non-null   object
dtypes: int64(2), object(11)
memory usage: 224.3+ KB


## Extracting `PUBLISH_YEAR` from `RELEASE_DATE`

In [26]:
df['RELEASE_DATE'].head(10)

0     15 January 2016 – 16:48:00 UTC (9 years ago)
1     9 November 2021 – 04:58:00 UTC (3 years ago)
2    26 November 2021 – 10:54:48 UTC (3 years ago)
3    15 February 2016 – 21:53:00 UTC (8 years ago)
4       25 May 2023 – 10:22:41 UTC (12 months ago)
5          8 May 2022 – 11:32:26 UTC (2 years ago)
6      31 July 2023 – 07:27:28 UTC (10 months ago)
7      17 April 2014 – 17:00:00 UTC (10 years ago)
8    13 December 2022 – 08:00:00 UTC (2 years ago)
9         7 July 2015 – 06:59:00 UTC (9 years ago)
Name: RELEASE_DATE, dtype: object

In [27]:
if 'RELEASE_DATE' in df.columns:
    df['PUBLISH_YEAR'] = df['RELEASE_DATE'].str.extract(r'(\d{4})')

    df['PUBLISH_YEAR'] = pd.to_numeric(df['PUBLISH_YEAR'], errors='coerce')

    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].fillna(-1)
    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].astype(int)
    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].replace(-1, pd.NA)

    df = df.drop('RELEASE_DATE', axis=1)

## Handle Missing Values:(`PUBLISH_YEAR`)

In [28]:
null_count_before = df['PUBLISH_YEAR'].isnull().sum()
null_count_before

4

In [29]:
mode_publish_year = df['PUBLISH_YEAR'].mode()
print(mode_publish_year[0])

2023


In [30]:
df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].fillna(mode_publish_year[0])

C:\Users\AFRAA\AppData\Local\Temp\ipykernel_16128\3436135746.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].fillna(mode_publish_year[0])


In [31]:
null_count_after = df['PUBLISH_YEAR'].isnull().sum()
null_count_after

0

In [32]:
df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].astype(int)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2051 entries, 0 to 2258
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   NAME                   2051 non-null   object
 1   STORE_GENRE            2047 non-null   object
 2   RATING_SCORE           2051 non-null   object
 3   N_SUPPORTED_LANGUAGES  2051 non-null   object
 4   DEVELOPERS             2050 non-null   object
 5   SUPPORTED_PLATFORMS    2003 non-null   object
 6   POSITIVE_REVIEWS       2051 non-null   int64 
 7   NEGATIVE_REVIEWS       2051 non-null   int64 
 8   TECHNOLOGIES           1673 non-null   object
 9   TOTAL_TWITCH_PEAK      2051 non-null   object
 10  PRICE                  1374 non-null   object
 11  24_HOUR_PEAK           2036 non-null   object
 12  PUBLISH_YEAR           2051 non-null   int32 
dtypes: int32(1), int64(2), object(10)
memory usage: 216.3+ KB


## Handle Missing Values:(`DEVELOPERS`)

In [34]:
df['DEVELOPERS'].isnull().sum()

1

In [35]:
df.dropna(subset=['DEVELOPERS'], inplace=True)

## Data Cleaning and Integer Conversion: (`N_SUPPORTED_LANGUAGES`)

This section handles the `N_SUPPORTED_LANGUAGES` column by filling missing values with a default number (1) and converting the column to an integer type after removing any extraneous text.

In [36]:
df['N_SUPPORTED_LANGUAGES'].isnull().sum()

0

In [37]:
df['N_SUPPORTED_LANGUAGES'].head()

0     7 Languages
1    24 Languages
2     1 Languages
3    12 Languages
4     1 Languages
Name: N_SUPPORTED_LANGUAGES, dtype: object

In [38]:
df['N_SUPPORTED_LANGUAGES'] = df['N_SUPPORTED_LANGUAGES'].fillna(1)

In [39]:
df['N_SUPPORTED_LANGUAGES'] = df['N_SUPPORTED_LANGUAGES'].astype(str).str.replace(' Languages', '').astype(int)

## Data Cleaning and Integer Conversion: (`RATING_SCORE`)

In [40]:
df['RATING_SCORE'].isnull().sum()

0

In [41]:
df['RATING_SCORE'].head()

0    review_score 8
1    review_score 8
2    review_score 8
3    review_score 8
4    review_score 8
Name: RATING_SCORE, dtype: object

In [42]:
placeholder_value = 'review_score -1'
df['RATING_SCORE'] = df['RATING_SCORE'].fillna(placeholder_value)

In [43]:
df['RATING_SCORE'] = df['RATING_SCORE'].str.replace('review_score ', '').astype(float).astype(int)  

In [44]:
# mean_rating_score = df.loc[df['RATING_SCORE'] != -1, 'RATING_SCORE'].mean()  
# df.loc[df['RATING_SCORE'] == -1, 'RATING_SCORE'] = mean_rating_score

## Data Cleaning and Transforming: (`STORE_GENRE`)

In [45]:
df['STORE_GENRE'].head(10)

0     Store Genres Action (1), Adventure (25), RPG (3)
1    Store Genres Action (1), Adventure (25), Racin...
2    Store Genres Casual (4), Indie (23), Simulatio...
3              Store Genres Adventure (25), Indie (23)
4           Store Genres Indie (23), Free to Play (37)
5    Store Genres Adventure (25), Casual (4), Free ...
6    Store Genres Action (1), Simulation (28), Stra...
7    Store Genres Indie (23), Simulation (28), Stra...
8              Store Genres Action (1), Adventure (25)
9    Store Genres Action (1), Indie (23), Racing (9...
Name: STORE_GENRE, dtype: object

In [46]:
def clean_store_genre(genre_string):
    genres = genre_string.split(', ')
    cleaned_genres = []
    for genre in genres:
        cleaned_genre = genre.split(' (')[0]
        cleaned_genres.append(cleaned_genre)
    return cleaned_genres

In [47]:
df['STORE_GENRE'] = df['STORE_GENRE'].fillna('Store Genres')
df['STORE_GENRE'] = df['STORE_GENRE'].str.replace('Store Genres ', '')
df['STORE_GENRE'] = df['STORE_GENRE'].apply(clean_store_genre)

In [48]:
df['STORE_GENRE'].head(10)

0                           [Action, Adventure, RPG]
1    [Action, Adventure, Racing, Simulation, Sports]
2          [Casual, Indie, Simulation, Early Access]
3                                 [Adventure, Indie]
4                              [Indie, Free to Play]
5                  [Adventure, Casual, Free to Play]
6                     [Action, Simulation, Strategy]
7                      [Indie, Simulation, Strategy]
8                                [Action, Adventure]
9                    [Action, Indie, Racing, Sports]
Name: STORE_GENRE, dtype: object

## Extracting and Handling `TWITCH_PEAK` Data

##### Extracting `TWITCH_PEAK_HOUR` & `TWITCH_PEAK_YEAR` from `TOTAL_TWITCH_PEAK`
##### Handle Missing Values:(`TWITCH_PEAK_YEAR`, `TWITCH_PEAK_HOUR`, `TWITCH_PEAK_HOUR`)

In [49]:
df['TOTAL_TWITCH_PEAK'].head()

0     27,368\nall-time peak 9 years ago17 January 2016
1    81,096\nall-time peak 3 years ago13 November 2021
2           145\nall-time peak 2 years ago24 July 2022
3       145,384\nall-time peak 6 years ago13 June 2018
4       3,778\nall-time peak 10 months ago18 July 2023
Name: TOTAL_TWITCH_PEAK, dtype: object

In [50]:
df['24_HOUR_PEAK'].isnull().sum()

15

In [51]:
if 'TOTAL_TWITCH_PEAK' in df.columns:
    # Extract and convert the TWITCH_PEAK_HOUR
    df['TWITCH_PEAK_HOUR'] = df['TOTAL_TWITCH_PEAK'].apply(lambda x: x.split('\n')[0].replace(',', ''))
    df['TWITCH_PEAK_HOUR'] = pd.to_numeric(df['TWITCH_PEAK_HOUR'], errors='coerce')
    df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].fillna(-1)
    df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].astype(int)
    df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].replace(-1, pd.NA)
    
    # Extract and convert the TWITCH_PEAK_YEAR
    df['TWITCH_PEAK_YEAR'] = df['TOTAL_TWITCH_PEAK'].apply(lambda x: x.split()[-1])
    df['TWITCH_PEAK_YEAR'] = pd.to_numeric(df['TWITCH_PEAK_YEAR'], errors='coerce')
    df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].fillna(-1)
    df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].astype(int)
    df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].replace(-1, pd.NA)



In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2050 entries, 0 to 2258
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   NAME                   2050 non-null   object
 1   STORE_GENRE            2050 non-null   object
 2   RATING_SCORE           2050 non-null   int32 
 3   N_SUPPORTED_LANGUAGES  2050 non-null   int32 
 4   DEVELOPERS             2050 non-null   object
 5   SUPPORTED_PLATFORMS    2002 non-null   object
 6   POSITIVE_REVIEWS       2050 non-null   int64 
 7   NEGATIVE_REVIEWS       2050 non-null   int64 
 8   TECHNOLOGIES           1672 non-null   object
 9   TOTAL_TWITCH_PEAK      2050 non-null   object
 10  PRICE                  1373 non-null   object
 11  24_HOUR_PEAK           2035 non-null   object
 12  PUBLISH_YEAR           2050 non-null   int32 
 13  TWITCH_PEAK_HOUR       2035 non-null   object
 14  TWITCH_PEAK_YEAR       2035 non-null   object
dtypes: int32(3), int64(2), obj

In [53]:
df['TWITCH_PEAK_YEAR'].isnull().sum()

15

In [54]:
df['TWITCH_PEAK_HOUR'].isnull().sum()

15

In [55]:
df[(df['TWITCH_PEAK_HOUR'].isnull() | df['TWITCH_PEAK_YEAR'].isnull())][['TWITCH_PEAK_HOUR','TWITCH_PEAK_YEAR', 'TOTAL_TWITCH_PEAK']]


,TWITCH_PEAK_HOUR,TWITCH_PEAK_YEAR,TOTAL_TWITCH_PEAK
5,<NA>,<NA>,2 years ago
10,<NA>,<NA>,5 years ago
122,<NA>,<NA>,9 years ago
152,<NA>,<NA>,4 years ago
279,<NA>,<NA>,5 months ago
286,<NA>,<NA>,6 years ago
539,<NA>,<NA>,4 years ago
548,<NA>,<NA>,13 months ago
572,<NA>,<NA>,11 days ago
580,<NA>,<NA>,14 months ago


In [56]:
data_collection_date = datetime(2024, 5, 28)

def calculate_twitch_peak_year(total_twitch_peak):
    match = re.match(r"(\d+)\s(\w+)\sago", total_twitch_peak)
    if not match:
        return None
    
    value, unit = int(match.group(1)), match.group(2)
    if unit.startswith('year'):
        delta = timedelta(days=365 * value)
    elif unit.startswith('month'):
        delta = timedelta(days=30 * value)
    elif unit.startswith('day'):
        delta = timedelta(days=value)
    else:
        return None
    
    peak_date = data_collection_date - delta
    return peak_date.year

In [57]:
df.loc[df['TWITCH_PEAK_YEAR'].isnull(), 'TWITCH_PEAK_YEAR'] = df.loc[df['TWITCH_PEAK_YEAR'].isnull(), 'TOTAL_TWITCH_PEAK'].apply(calculate_twitch_peak_year)

In [58]:
df[(df['TWITCH_PEAK_HOUR'].isnull() | df['TWITCH_PEAK_YEAR'].isnull())][['TWITCH_PEAK_HOUR','TWITCH_PEAK_YEAR', 'TOTAL_TWITCH_PEAK']]


,TWITCH_PEAK_HOUR,TWITCH_PEAK_YEAR,TOTAL_TWITCH_PEAK
5,<NA>,2022,2 years ago
10,<NA>,2019,5 years ago
122,<NA>,2015,9 years ago
152,<NA>,2020,4 years ago
279,<NA>,2023,5 months ago
286,<NA>,2018,6 years ago
539,<NA>,2020,4 years ago
548,<NA>,2023,13 months ago
572,<NA>,2024,11 days ago
580,<NA>,2023,14 months ago


In [59]:
df[(df['TWITCH_PEAK_HOUR'].isnull() |  df['TWITCH_PEAK_YEAR'].isnull())][['TWITCH_PEAK_HOUR', '24_HOUR_PEAK']]

,TWITCH_PEAK_HOUR,24_HOUR_PEAK
5,<NA>,NaN
10,<NA>,NaN
122,<NA>,NaN
152,<NA>,NaN
279,<NA>,NaN
286,<NA>,NaN
539,<NA>,NaN
548,<NA>,NaN
572,<NA>,NaN
580,<NA>,NaN


In [60]:
df = df.dropna(subset=['TWITCH_PEAK_HOUR', '24_HOUR_PEAK'], how='all')

In [61]:
#Drop the original column `TOTAL_TWITCH_PEAK`
df.drop(columns=['TOTAL_TWITCH_PEAK'], inplace=True)

In [62]:
df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].astype(float)
df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].astype(int)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2035 entries, 0 to 2258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2035 non-null   object 
 1   STORE_GENRE            2035 non-null   object 
 2   RATING_SCORE           2035 non-null   int32  
 3   N_SUPPORTED_LANGUAGES  2035 non-null   int32  
 4   DEVELOPERS             2035 non-null   object 
 5   SUPPORTED_PLATFORMS    1988 non-null   object 
 6   POSITIVE_REVIEWS       2035 non-null   int64  
 7   NEGATIVE_REVIEWS       2035 non-null   int64  
 8   TECHNOLOGIES           1658 non-null   object 
 9   PRICE                  1371 non-null   object 
 10  24_HOUR_PEAK           2035 non-null   object 
 11  PUBLISH_YEAR           2035 non-null   int32  
 12  TWITCH_PEAK_HOUR       2035 non-null   float64
 13  TWITCH_PEAK_YEAR       2035 non-null   int32  
dtypes: float64(1), int32(4), int64(2), object(7)
memory usage: 20

## Transforming: (`24_HOUR_PEAK`)


In [64]:
df['24_HOUR_PEAK'].head()

0     1,064\n24-hour peak
1    16,131\n24-hour peak
2        21\n24-hour peak
3        31\n24-hour peak
4       318\n24-hour peak
Name: 24_HOUR_PEAK, dtype: object

In [65]:
df['24_HOUR_PEAK'] = df['24_HOUR_PEAK'].fillna(("-1\n24-hour peak"))
df['24_HOUR_PEAK'] = df['24_HOUR_PEAK'].apply(lambda x: int(x.split('\n')[0].replace(',', ''))).astype(float)

In [66]:
df.loc[df['24_HOUR_PEAK'] == -1, '24_HOUR_PEAK'] = pd.NA

In [67]:
df['24_HOUR_PEAK'].describe()

count      2035.000000
mean       1184.959214
std        6437.130999
min           0.000000
25%          10.000000
50%          37.000000
75%         210.000000
max      109950.000000
Name: 24_HOUR_PEAK, dtype: float64

In [68]:
df['24_HOUR_PEAK'].isnull().sum()

0

In [69]:
df['24_HOUR_PEAK'] = df['24_HOUR_PEAK'].fillna(df['24_HOUR_PEAK'].median())

## Extracting `TOTAL_REVIEW` from `POSITIVE_REVIEWS` & `NEGATIVE_REVIEWS`

This section calculates a new 'TOTAL_REVIEW' column, which represents the proportion of positive reviews out of the total number of reviews (positive and negative). This helps in understanding the overall review distribution for each game.

In [70]:
df['POSITIVE_REVIEWS'].isnull().sum()

0

In [71]:
df['NEGATIVE_REVIEWS'].isnull().sum()

0

In [72]:
df['TOTAL_REVIEW'] = df['POSITIVE_REVIEWS'] / (df['NEGATIVE_REVIEWS'] + df['POSITIVE_REVIEWS'])
df['TOTAL_REVIEW'].head()

0    0.886759
1    0.881440
2    0.925690
3    0.889818
4    0.926667
Name: TOTAL_REVIEW, dtype: float64


## <span style="color:RED"> Cleaning and Handling Missing: (`TECHNOLOGIES`) </span>

In [73]:
df['TECHNOLOGIES'].head(10)

0                                                   NaN
1                                                   NaN
2                                          Engine.Godot
3                                          Engine.Unity
4          Detected Technologies (?), SDK.SteamworksNET
6                                          Engine.Unity
7                                                   NaN
8                 Engine.Unreal, SDK.EpicOnlineServices
9                 Engine.Unreal, SDK.EpicOnlineServices
11    Engine.PyGame, Engine.RenPy, Detected Technolo...
Name: TECHNOLOGIES, dtype: object

In [74]:
df['TECHNOLOGIES'] = df['TECHNOLOGIES'].str.replace('Detected Technologies (?)', '', regex=False).str.replace(', ,', ',').str.strip(', ')

In [75]:
df['TECHNOLOGIES'].head(10)

0                                                   NaN
1                                                   NaN
2                                          Engine.Godot
3                                          Engine.Unity
4                                     SDK.SteamworksNET
6                                          Engine.Unity
7                                                   NaN
8                 Engine.Unreal, SDK.EpicOnlineServices
9                 Engine.Unreal, SDK.EpicOnlineServices
11    Engine.PyGame, Engine.RenPy, Engine.PyGame, En...
Name: TECHNOLOGIES, dtype: object

In [76]:
df[df['TECHNOLOGIES'].isnull()][['NAME', 'PUBLISH_YEAR']].head(10)

,NAME,PUBLISH_YEAR
0,Dragon's Dogma: Dark Arisen,2016
1,Forza Horizon 5,2021
7,Wargame: Red Dragon,2014
23,Ghost of Tsushima DIRECTOR'S CUT,2024
36,100 hidden dogs,2021
40,Rush Rally 3,2022
44,100 hidden birds,2020
62,Granblue Fantasy: Relink,2024
66,Cursed 3,2022
68,Stardom 3 Deluxe,2022


In [77]:
df['TECHNOLOGIES'] = df['TECHNOLOGIES'].fillna(' ')  # Replace NaN with empty string
df['TECHNOLOGIES'] = df['TECHNOLOGIES'].apply(lambda x: x.split(', ') if x else [])

In [78]:
df['TECHNOLOGIES']

0                                         [ ]
1                                         [ ]
2                              [Engine.Godot]
3                              [Engine.Unity]
4                         [SDK.SteamworksNET]
                        ...                  
2253                           [Engine.Unity]
2254                                      [ ]
2256                         [Engine.Solar2D]
2257    [AntiCheat.PunkBuster, Engine.idTech]
2258                           [Engine.Unity]
Name: TECHNOLOGIES, Length: 2035, dtype: object

## <span style="color:RED"> One-Hot Encoding `TECHNOLOGIES` Column (`TECHNOLOGIES`) </span>

In [79]:
## ##################################################TODO


## <span style="color:RED"> One-Hot Encoding `STORE_GENRE` Column (`STORE_GENRE`) </span>

In [80]:
## ##################################################TODO

## One-Hot Encoding `SUPPORTED_PLATFORMS` Column

This section transforms the 'SUPPORTED_PLATFORMS' column into separate binary columns for each platform using one-hot encoding, which facilitates easier analysis and modeling.

In [81]:
df['SUPPORTED_PLATFORMS']

0                   Windows
1                   Windows
2             Windows Linux
3       Windows macOS Linux
4                   Windows
               ...         
2253          Windows macOS
2254          Windows macOS
2256          Windows macOS
2257                    NaN
2258                Windows
Name: SUPPORTED_PLATFORMS, Length: 2035, dtype: object

In [82]:
df['SUPPORTED_PLATFORMS'].isnull().sum()

47

In [83]:
# all_platforms = df['SUPPORTED_PLATFORMS'].dropna().str.split().sum()
# platform_counts = Counter(all_platforms)
# most_common_platform = platform_counts.most_common(1)[0][0]

# most_common_platform

# df.loc[df['SUPPORTED_PLATFORMS'].isnull(), 'SUPPORTED_PLATFORMS'] = most_common_platform

In [84]:
# Split the 'SUPPORTED_PLATFORMS' column into separate columns for each platform
platform_dummies = df['SUPPORTED_PLATFORMS'].str.get_dummies(sep=' ')

# Concatenate the dummy columns with the original DataFrame
df = pd.concat([df, platform_dummies], axis=1)

In [85]:
df.drop(columns=['SUPPORTED_PLATFORMS'], inplace=True)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2035 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2035 non-null   object 
 1   STORE_GENRE            2035 non-null   object 
 2   RATING_SCORE           2035 non-null   int32  
 3   N_SUPPORTED_LANGUAGES  2035 non-null   int32  
 4   DEVELOPERS             2035 non-null   object 
 5   POSITIVE_REVIEWS       2035 non-null   int64  
 6   NEGATIVE_REVIEWS       2035 non-null   int64  
 7   TECHNOLOGIES           2035 non-null   object 
 8   PRICE                  1371 non-null   object 
 9   24_HOUR_PEAK           2035 non-null   float64
 10  PUBLISH_YEAR           2035 non-null   int32  
 11  TWITCH_PEAK_HOUR       2035 non-null   float64
 12  TWITCH_PEAK_YEAR       2035 non-null   int32  
 13  TOTAL_REVIEW           2035 non-null   float64
 14  Linux                  2035 non-null   int64  
 15  Windows  

## Merging Game Information and Price DataFrames

### Removing `PRICE` Column

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2035 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2035 non-null   object 
 1   STORE_GENRE            2035 non-null   object 
 2   RATING_SCORE           2035 non-null   int32  
 3   N_SUPPORTED_LANGUAGES  2035 non-null   int32  
 4   DEVELOPERS             2035 non-null   object 
 5   POSITIVE_REVIEWS       2035 non-null   int64  
 6   NEGATIVE_REVIEWS       2035 non-null   int64  
 7   TECHNOLOGIES           2035 non-null   object 
 8   PRICE                  1371 non-null   object 
 9   24_HOUR_PEAK           2035 non-null   float64
 10  PUBLISH_YEAR           2035 non-null   int32  
 11  TWITCH_PEAK_HOUR       2035 non-null   float64
 12  TWITCH_PEAK_YEAR       2035 non-null   int32  
 13  TOTAL_REVIEW           2035 non-null   float64
 14  Linux                  2035 non-null   int64  
 15  Windows  

In [88]:
game_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4950 entries, 0 to 4949
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     4950 non-null   object
 1   url      4950 non-null   object
 2   price    4950 non-null   object
 3   release  4950 non-null   object
 4   page     4950 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 193.5+ KB


In [89]:
non_numeric = game_details[~game_details['release'].str.isnumeric()]
print("Non-numeric values:\n", non_numeric)
game_details_cleaned = game_details[game_details['release'].str.isnumeric()]

game_details['release'] = pd.to_numeric(game_details['release'], errors='coerce')

game_details['release'] = game_details['release'].astype('Int64')

Non-numeric values:
                                   name                                url  \
352             Building Relationships  https://steamdb.info/app/2666920/   
354          BoneField: Bodycam Horror  https://steamdb.info/app/2567770/   
355                         TEXNOPLAZM  https://steamdb.info/app/2527710/   
356                Ripple In Dirac Sea  https://steamdb.info/app/2421650/   
357                     100 Egypt Cats  https://steamdb.info/app/3017380/   
...                                ...                                ...   
2749                        Splinterra  https://steamdb.info/app/2779600/   
3225             Football Manager 2011    https://steamdb.info/app/34220/   
3782  Assassin’s Creed® IV Black Flag™   https://steamdb.info/app/242050/   
4487    Serious Sam Fusion 2017 (beta)   https://steamdb.info/app/564310/   
4772                   Sniper Elite V2    https://steamdb.info/app/63380/   

        price release  page  
352         -       -   

In [90]:
def standardize_spaces(text):
    if isinstance(text, str):
        return re.sub(r'\s+', ' ', text)
    return text

df['NAME'] = df['NAME'].astype(str).str.lower().str.strip().apply(standardize_spaces)
game_details['name'] = game_details['name'].astype(str).str.lower().str.strip().apply(standardize_spaces)

In [91]:
def is_valid_price(price):
    return not (str(price).startswith('$') or str(price) == '-')

df.loc[~df['PRICE'].apply(is_valid_price), 'PRICE'] = np.nan
filtered_df = df[df['PRICE'].apply(is_valid_price)]

for index, row in filtered_df.iterrows():
    game_name = row['NAME']
    game_date = row['PUBLISH_YEAR']
    price = game_details[game_details['name'] == game_name]['price'].values
    
    if len(price) > 0:
        df.at[index, 'PRICE'] = price[0]

In [92]:
# name_set = set(df['NAME'])

# missing_games_urls = game_details[~game_details['name'].isin(name_set)]['url'].tolist()

# with open('missing_games_urls.txt', 'w') as file:
#     for url in missing_games_urls:
#         file.write(url + '\n')

In [93]:
df[df['PRICE'].isnull()]

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,PRICE,24_HOUR_PEAK,PUBLISH_YEAR,TWITCH_PEAK_HOUR,TWITCH_PEAK_YEAR,TOTAL_REVIEW,Linux,Windows,macOS
0,dragon's dogma: dark arisen,"[Action, Adventure, RPG]",8,7,Capcom,39271,5015,[ ],NaN,1064.0,2016,27368.0,2016,0.886759,0,1,0
1,forza horizon 5,"[Action, Adventure, Racing, Simulation, Sports]",8,24,Playground Games,156683,21075,[ ],NaN,16131.0,2021,81096.0,2021,0.881440,0,1,0
3,layers of fear (2016),"[Adventure, Indie]",8,12,Bloober Team SA,22766,2819,[Engine.Unity],NaN,31.0,2016,145384.0,2018,0.889818,1,1,1
4,taskpals,"[Indie, Free to Play]",8,1,lazarche,834,66,[SDK.SteamworksNET],NaN,318.0,2023,3778.0,2023,0.926667,0,1,0
7,wargame: red dragon,"[Indie, Simulation, Strategy]",8,11,Eugen Systems,16273,1969,[ ],NaN,802.0,2014,4779.0,2014,0.892062,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2253,mad games tycoon,"[Casual, Indie, Simulation, Strategy]",8,18,Eggcode,3988,403,[Engine.Unity],NaN,17.0,2016,1434.0,2016,0.908221,0,1,1
2254,fireboy & watergirl: elements,[Action],8,1,Oslo Albet,707,93,[ ],NaN,13.0,2019,118.0,2020,0.883750,0,1,1
2256,coromon,"[Adventure, Indie, RPG, Strategy]",8,18,TRAGsoft,4355,672,[Engine.Solar2D],NaN,254.0,2022,4026.0,2022,0.866322,0,1,1
2257,call of duty: united offensive,[Action],8,3,Gray Matter Studios,1648,247,"[AntiCheat.PunkBuster, Engine.idTech]",NaN,61.0,2006,96.0,2020,0.869657,0,0,0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2035 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2035 non-null   object 
 1   STORE_GENRE            2035 non-null   object 
 2   RATING_SCORE           2035 non-null   int32  
 3   N_SUPPORTED_LANGUAGES  2035 non-null   int32  
 4   DEVELOPERS             2035 non-null   object 
 5   POSITIVE_REVIEWS       2035 non-null   int64  
 6   NEGATIVE_REVIEWS       2035 non-null   int64  
 7   TECHNOLOGIES           2035 non-null   object 
 8   PRICE                  1326 non-null   object 
 9   24_HOUR_PEAK           2035 non-null   float64
 10  PUBLISH_YEAR           2035 non-null   int32  
 11  TWITCH_PEAK_HOUR       2035 non-null   float64
 12  TWITCH_PEAK_YEAR       2035 non-null   int32  
 13  TOTAL_REVIEW           2035 non-null   float64
 14  Linux                  2035 non-null   int64  
 15  Windows  

In [95]:
# import time
# import random
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.edge.options import Options
# from webdriver_manager.microsoft import EdgeChromiumDriverManager
# from urllib.parse import quote

# def get_game_price(driver, game_name):
#     time.sleep(random.uniform(5, 10)) 
#     search_url = f"https://steamdb.info/instantsearch/?query={quote(game_name)}"
#     print(game_name, search_url)
#     driver.get(search_url)
#     print(search_url)

#     # if "Please verify" in driver.page_source:
#     #     #input("Complete the verification and press Enter to continue...")
#     #     #time.sleep(random.uniform(5, 10)) 
#     #     return None
    
#     hits = driver.find_elements(By.CLASS_NAME, 'ais-Hits-item')
#     for hit in hits:
#         hit_name = hit.find_element(By.CLASS_NAME, 's-hit--name').text.strip()
#         if hit_name.lower() == game_name.lower():
#             try:
#                 price = hit.find_element(By.CLASS_NAME, 's-hit--price').text.strip()
#                 return price
#             except:
#                 return '-'
#     return None  


# try:
#     existing_prices_df = pd.read_csv('games_missing_price.csv')
#     existing_prices = dict(zip(existing_prices_df['NAME'], existing_prices_df['PRICE']))
# except FileNotFoundError:
#     existing_prices = {}

# options = Options()
# options.use_chromium = True
# driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)

# games_missing_price = df[df['PRICE'].isnull()]
# max_attempts = 5
# attempts = 0

# while not games_missing_price.empty and attempts < max_attempts:
#     attempts += 1
#     print(f"Attempt {attempts}")

#     games_missing_price = games_missing_price.sample(frac=1).reset_index(drop=True)

#     for index, row in games_missing_price.iterrows():
#         game_name = row['NAME']
#         if game_name in existing_prices:
#             price = existing_prices[game_name]
#             print(f"Found existing price for {game_name}: {price}")
#         else:
#             price = get_game_price(driver, game_name)
#             if price != None:
#                 df.at[index, 'PRICE'] = price
#                 existing_prices[game_name] = price
#                 print(f"Updated price for {game_name}: {price}")
#             else:
#                 print(f"Price not found for {game_name}")

        

#     games_missing_price = df[df['PRICE'].isnull()]

# prices_df = pd.DataFrame(list(existing_prices.items()), columns=['NAME', 'PRICE'])
# prices_df.to_csv('games_missing_price.csv', index=False)

# driver.quit()


In [96]:
null_count = df['PRICE'].isnull().sum()

print("Null count in PRICE column:", null_count)

Null count in PRICE column: 709


In [97]:
del game_details

## Handling 'PRICE' Column

In [98]:
df['PRICE'] = df['PRICE'].replace('-', '$0')

In [99]:
price_pattern = r'\$([\d.]+)'

df['PRICE'] = df['PRICE'].str.extract(price_pattern)

In [100]:
df['PRICE'] = df['PRICE'].astype(float)

In [101]:
null_count = df['PRICE'].isnull().sum()

print("Null count in PRICE column:", null_count)

Null count in PRICE column: 777


In [102]:
df['PRICE'].describe()

count    1258.00000
mean       13.10287
std        12.24461
min         0.00000
25%         3.04000
50%         9.99000
75%        19.99000
max        69.99000
Name: PRICE, dtype: float64

In [103]:
# Drop rows with null values in the 'PRICE' column
#df.dropna(subset=['PRICE'], inplace=True)
#df['PRICE'] = df['PRICE'].fillna('$0')

In [104]:
# df[df['PRICE'].isnull() == True]

In [105]:
# null_count = df['PRICE'].isnull().sum()

# print("Null count in PRICE column:", null_count)

## Final Dataset Preparation

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2035 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2035 non-null   object 
 1   STORE_GENRE            2035 non-null   object 
 2   RATING_SCORE           2035 non-null   int32  
 3   N_SUPPORTED_LANGUAGES  2035 non-null   int32  
 4   DEVELOPERS             2035 non-null   object 
 5   POSITIVE_REVIEWS       2035 non-null   int64  
 6   NEGATIVE_REVIEWS       2035 non-null   int64  
 7   TECHNOLOGIES           2035 non-null   object 
 8   PRICE                  1258 non-null   float64
 9   24_HOUR_PEAK           2035 non-null   float64
 10  PUBLISH_YEAR           2035 non-null   int32  
 11  TWITCH_PEAK_HOUR       2035 non-null   float64
 12  TWITCH_PEAK_YEAR       2035 non-null   int32  
 13  TOTAL_REVIEW           2035 non-null   float64
 14  Linux                  2035 non-null   int64  
 15  Windows  

In [107]:
df.to_csv('preprocessed_game_info.csv', index=False)